# Random forest

In [1]:
import numpy as np
import pandas as pd
import operator as op

In [2]:
x_train = np.array([[1,1,1],[1,1,0],[0,1,1],[0,0,0]])
y_train = np.array([1,1,0,0])
x_test = np.array([[1,0,1],[0,1,0]])
y_train[[0,2]]
len(x_test)

2

In [3]:
class Random_forest :
    def __init__(self):
        self.is_leaf = False
        self.stumps = [[[0,2],[0,2]],[[1,3],[0,1]],[[0,3],[1,2]]]
        #self.stump_2 = [[1,3],[0,1]]
        #self.stump_3 = [[0,4],[1,3]]
        
    def __call__(self , x_data):
        if self.is_leaf :
            result = self.output_class 
            return result
        else :
            cont_out = np.empty(len(x_data))
            cont_out.fill(np.nan)
            div = self.rule(x_data)
            cont_out[div] = self.left(x_data[div])          
            cont_out[op.invert(div)] = self.right(x_data[op.invert(div)])
        return cont_out
    
    def split_rule (self,col , row):
        rule = lambda ru: ru[:,col] < row
        return rule
    
    def separate_stump(self,stump,x_data,y_data):
        x_data1 = x_data[stump[0],:][:,stump[1]]
        y_data1 = y_data[stump[0]]
        return x_data1,y_data1
    
    def stop(self,X, Y):
        if len(Y) <= 1 or len(np.unique(Y)) <= 1:
            return True
        
        return False
    
    def fit_bagging(self, x_data,y_data):
        self.trees = {}
        for i,stump in zip(range(len(self.stumps)), self.stumps):
            self.trees["self.tree{}".format(i)] = self.spliter(self.separate_stump(stump,x_data,y_data)[0],
                                                        self.separate_stump(stump,x_data,y_data)[1])
        print("The trained trees {}".format(self.trees))
        return self
        
    def predict_bags(self,test):
        pred = []
        for i in range(3):
            prediction = self.trees["self.tree{}".format(i)](test)
            pred.append(prediction)
            print("prediction for stump {}, tree{}: {}".format(i+1,i, prediction))
        res1 = []
        res2 = []
        for i in pred:
            res1.append(i[0])
            res2.append(i[1])
        res11 = max(set(res1),key = res1.count)
        res22 = max(set(res2),key = res1.count)
        result = np.array([res11,res22]).astype("int32")
        return result
    
    def spliter (self , x_data, y_data):
        if self.stop(x_data, y_data):
            self.is_leaf = True
            unique_classes = np.unique(y_data)
            _, counter = np.unique(y_data, return_counts = True )
            mode = unique_classes[np.argmax(counter)]
            self.output_class = mode
            return self
        rowss, columns = x_data. shape
        outcome_ = np.empty([rowss, columns])
        outcome_.fill(np.nan)
        for ro in range (rowss): 
            for co in range (columns):
                s_num = x_data[:, co] < x_data[ro, co]
                outcome_[ro, co] = self.condition_split(s_num , x_data)
        
        ro,co = np.unravel_index(np. argmax (outcome_, axis = None ), outcome_.shape )
        self.rule =self.split_rule (co, x_data[ro,co])
        s_num =self.rule(x_data)
        self.left = Random_forest().spliter(x_data[s_num], y_data[s_num])
        self.right = Random_forest().spliter(x_data[op.invert(s_num)], y_data[op.invert(s_num)])
        self.is_leaf = False
        return self
    
    
    def condition_split(self,split_num, y_data):
        if sum(split_num) in {0, len(y_data)}:
            return 0 
        unique_classes = np.unique(y_data)
        cont = np.empty([2 , len(unique_classes)])
        cont.fill(np.nan)
        for i, js in enumerate(unique_classes):
            cont[0, i] = np.sum(y_data[split_num]== js)
            cont[1, i] = np.sum(y_data[op.invert(split_num)]== js)
        r_sum = np.sum(cont, axis =1)
        rr_sum = np.sum(cont, axis =0)
        _sum = np.sum(cont)
        first = -np.sum(np.divide(rr_sum,_sum)**2)
        second = +np.sum(np.divide(r_sum,_sum)*np.sum(np.divide(cont,r_sum[:,None])**2 , axis =1) )
        final = np.add(first,second)
        return final
     
        

In [4]:
model = Random_forest()
model.fit_bagging(x_train , y_train )
prediction = model.predict_bags(x_test)
print("The Random forest prediction for row 5 is '{}' while the prediction for row 6 is '{}'".format(prediction[0],prediction[1]))

The trained trees {'self.tree0': <__main__.Random_forest object at 0x0000021391741F88>, 'self.tree1': <__main__.Random_forest object at 0x0000021391741F88>, 'self.tree2': <__main__.Random_forest object at 0x0000021391741F88>}
prediction for stump 1, tree0: [1. 0.]
prediction for stump 2, tree1: [1. 0.]
prediction for stump 3, tree2: [1. 0.]
The Random forest prediction for row 5 is '1' while the prediction for row 6 is '0'
